To run nebula scirbt you need the follwing libraries:

```R
    library(glue)
    library(dplyr)
    library(getopt)
    library(Matrix)
    library(qs)
    library(Seurat)
    library(nebula)
    library(future)
```

```R
install.packages("getopt")

if (!require("devtools")) install.packages("devtools")
devtools::install_github("lhe17/nebula")

if (!require("BiocManager")) install.packages("BiocManager")
BiocManager::install("biomaRt")

```

R scritp from [link](https://github.com/bdferris642/sc-online/tree/main/scripts)

# Hyperparameters

In [1]:
# ATTENTION: need for perpty of running jupternotbook
%matplotlib inline

%load_ext autoreload
%autoreload 2



import pandas as pd
import numpy as np
from scipy import sparse
import scanpy as sc
import matplotlib.pyplot as plt
import os

from dotenv import load_dotenv; load_dotenv()


from utils import preprocessing
from utils import DEG


GRADIENT_GENES_CORR_PATH = os.getenv("GRADIENT_GENES_CORR_PATH")
PARALLEL_NEBULA_SCRIPT_PATH = os.getenv("PARALLEL_NEBULA_SCRIPT")

# name of cell tyoe vauble annotaton to use in this analsys
CT_FOR_DEG_VARIABLE = "Group_name" #"Group_name"
# name of Sample varibale
SAMPLE_VARIABLE = "donor_id"
# variable to test if differtially epxressed
CONTRAST_VARIABLE = "case_control"
# Level of contrat varibale to use as baseline
CONTRAST_BASELINE = "control"
# Level of contrat varibale to use as stimulated
CONTRAST_STIM = "case"
# Varibale combainstion to make groups (SAMPLE_VARIABLE non necessary)
COV_FOR_PSEUDOBULK = [CT_FOR_DEG_VARIABLE]
# Obs col names that groups psudocells in groups in which perfomr DEG in them --> it is made using the var in COV_FOR_PSEUDOBULK
GROUP_DEG_COL = "pseudobulk_group_for_DEG"

# tmp folder where to save intemrediate results
QS_TMP_FOLDER = "/home/gdallagl/myworkdir/XDP/data/_old/nebula_tmp"

# Covariates to use for stat test
COVARIATES_FOR_DEG = ["Age.at.Death", "Sex", "PMI", "pct_mt", "pct_intronic"]
# libraru size col for  enbula
LIBRARY_SIZE_COL = "nCount_RNA"
# Interesting cov (covarintes that are not used for the analys but still want to rpesevr in pesudobulk obs)
INTERESTING_COV = ["nCount_RNA", "nFeature_RNA", "pct_ribosomal", "library", "Cause.of.Death"]

# mitluple etst correction pvalue thr
ALPHA_MULTIPLE_TEST = 0.05
LOGFC_THR=0.1
PVAL_THR=0.05

# Read adata

In [2]:
adata = sc.read_h5ad(
    #"/home/gdallagl/myworkdir/XDP/data/XDP/NucSec/original_data/NucSeq_all_labelled_zoned.h5ad", 
    "/home/gdallagl/myworkdir/XDP/data/XDP/artificial_bican/geneset_001/zonated_objs_combined_with_md__combined__rep_001__ventral_matrix_keep_1.0_raw.h5ad",
    #backed="r"
    )

#adata = adata[adata.obs.spn_type == "dorsal_matrix"].copy()


adata

AnnData object with n_obs × n_vars = 191639 × 37905
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'donor_id', 'PREFIX', 'CELL_BARCODE', 'NUM_GENIC_READS', 'NUM_TRANSCRIPTS', 'NUM_GENES', 'num_retained_transcripts', 'pct_coding', 'pct_utr', 'pct_intergenic', 'pct_genic', 'pct_intronic', 'pct_mt', 'pct_ribosomal', 'frac_contamination', 'experiment', 'Neighborhood_name', 'Neighborhood_bootstrapping_probability', 'Class_name', 'Class_bootstrapping_probability', 'Subclass_name', 'Subclass_bootstrapping_probability', 'Group_name', 'Group_bootstrapping_probability', 'Cluster_name', 'Cluster_alias', 'Cluster_bootstrapping_probability', 'x', 'y', 'library', 'donor', 'x_um2', 'y_um2', 'unique_cell_ID', 'cb', 'Final_Zone_Assignments', 'Group_name.1', 'cell_type', 'Cohort', 'Biobank', 'Age.at.Death', 'Sex', 'Race', 'Ethnicity', 'Cause.of.Death', 'PMI', 'spn_type', 'case_control', 'RNA_snn_res.0.2', 'seurat_clusters', 'RNA_snn_res.0.5', 'Row.names', 'PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'P

In [ ]:
# adata_c = sc.read_h5ad("/home/gdallagl/myworkdir/XDP/data/XDP/diseased/sample_01/adata/all_lib_adata_zoned.h5ad")
# adata_c.var_names = adata_c.var.gene_symbol.astype(str)
# adata_c.var_names_make_unique() 

# Calculate gradient-score

Specific for cell type (all samples together).

In [4]:
def compute_gradient_score(adata, gradient_csv_path, 
                          p_threshold=0.05,
                          corr_threshold=None, 
                          use_layer=None,
                          standardize=True):
    """
    Compute dorsal-ventral gradient scores.
    
    Parameters:
    -----------
    adata : AnnData
        Single-cell data
    gradient_csv_path : str
        Path to gradient gene CSV
    p_threshold : float
        P-value threshold (default: 0.05)
    corr_threshold : float, optional
        Minimum absolute correlation (e.g., 0.1)
    use_layer : str or None
        'raw', 'counts', or None for adata.X
    standardize : bool
        Z-score normalize genes (recommended: True)
    
    Returns:
    --------
    dict with gradient_score, dorsal_score, ventral_score
    """
    # Load and filter gradient data
    gradient_df = pd.read_csv(gradient_csv_path)
    gradient_df = gradient_df[gradient_df['combined_p'] < p_threshold]
    
    if corr_threshold:
        gradient_df = gradient_df[np.abs(gradient_df['mean_spearman']) >= corr_threshold]
    
    gradient_df = gradient_df[gradient_df['gene'].isin(adata.var_names)]
    
    # Split dorsal/ventral
    dorsal_df = gradient_df[gradient_df['mean_spearman'] > 0]
    ventral_df = gradient_df[gradient_df['mean_spearman'] < 0]
    
    print(f"Using {len(gradient_df)} genes (p<{p_threshold}): "
          f"{len(dorsal_df)} dorsal, {len(ventral_df)} ventral")
    
    # Get expression matrix
    if use_layer == 'raw':
        X = adata.raw.X if adata.raw else adata.X
        var_names = adata.raw.var_names if adata.raw else adata.var_names
    elif use_layer:
        X = adata.layers[use_layer]
        var_names = adata.var_names
    else:
        X = adata.X
        var_names = adata.var_names
    
    # Helper function to compute score
    def compute_score(df, negate=False):
        if len(df) == 0:
            return np.zeros(X.shape[0])
        
        idx = [list(var_names).index(g) for g in df['gene']]
        X_sub = X[:, idx]
        
        if sparse.issparse(X_sub):
            X_sub = X_sub.toarray()
        
        if standardize:
            means = X_sub.mean(axis=0)
            stds = X_sub.std(axis=0)
            stds[stds == 0] = 1
            X_sub = (X_sub - means) / stds
        
        weights = df['mean_spearman'].values
        score = X_sub @ weights
        
        return -score if negate else score
    
    # Compute scores
    gradient_scores = compute_score(gradient_df)
    dorsal_scores = compute_score(dorsal_df)
    ventral_scores = compute_score(ventral_df, negate=True)
    
    # Store in adata
    adata.obs['gradient_score'] = gradient_scores
    adata.obs['dorsalness'] = gradient_scores
    adata.obs['dorsal_score'] = dorsal_scores
    adata.obs['ventral_score'] = ventral_scores
    
    return {
        'gradient_score': gradient_scores,
        'dorsal_score': dorsal_scores,
        'ventral_score': ventral_scores
    }


In [28]:
def save_files_for_R_conversion(adata, PATH_BASE):
    from scipy.io import mmwrite
    import pandas as pd
    import os
    
    os.makedirs(PATH_BASE, exist_ok=True)
    
    # Save sparse matrix (genes × cells)
    matrix_path = f"{PATH_BASE}/counts_matrix.mtx"
    mmwrite(matrix_path, adata.X.T)
    
    # Save gene names
    genes_path = f"{PATH_BASE}/genes.txt"
    with open(genes_path, 'w') as f:
        for gene in adata.var_names:
            f.write(f"{gene}\n")
    
    # Save cell/sample names
    barcodes_path = f"{PATH_BASE}/barcodes.txt"
    with open(barcodes_path, 'w') as f:
        for barcode in adata.obs_names:
            f.write(f"{barcode}\n")
    
    # Save metadata
    metadata_path = f"{PATH_BASE}/metadata.csv"
    adata.obs.to_csv(metadata_path)

def build_qs_from_python_files(PATH_BASE, output_qs_name="R_obj"):
    import rpy2.robjects as ro
    r = ro.r
    
    r(f'''
    library(Matrix)
    library(Seurat)
    library(qs)
    
    # Read sparse matrix
    mtx <- readMM("{PATH_BASE}/counts_matrix.mtx")
    
    # Read gene and sample names
    genes <- readLines("{PATH_BASE}/genes.txt")
    barcodes <- readLines("{PATH_BASE}/barcodes.txt")
    
    # Assign names
    rownames(mtx) <- genes
    colnames(mtx) <- barcodes
    
    # Read metadata
    metadata <- read.csv("{PATH_BASE}/metadata.csv", row.names=1)
    
    # Create Seurat object
    sobj <- CreateSeuratObject(counts = mtx, meta.data = metadata)
    
    # Save as .qs
    qsave(sobj, "{PATH_BASE}/{output_qs_name}.qs")
    ''')
    

In [ ]:
# Initialize X_umap if it doesn't exist
if 'X_umap' not in adata.obsm:
    adata.obsm['X_umap'] = np.full((adata.shape[0], 2), np.nan)

for ct in adata.obs[CT_FOR_DEG_VARIABLE].unique():

    a = adata[adata.obs[CT_FOR_DEG_VARIABLE] == ct].copy()

    print(ct,"\n", a)

    #############################
    # Save counts

    if "counts" in a.layers:
        print("Putting raw counts in .X")
        a.X = a.layers["counts"].copy()
    else:
        a.layers["counts"] = a.X.copy()
        print("Be sure that .X is raw counts")

    #############################
    # Compute gradients
    scores = compute_gradient_score(
        a,
        GRADIENT_GENES_CORR_PATH,
        use_layer="counts",
        standardize=True,
        p_threshold=0.05,
        corr_threshold=0.1
    )

    # SAVE BACK to original adata (only for these cells)
    adata.obs.loc[a.obs_names, 'gradient_score'] = a.obs['gradient_score']
    adata.obs.loc[a.obs_names, 'dorsal_score'] = a.obs['dorsal_score']
    adata.obs.loc[a.obs_names, 'ventral_score'] = a.obs['ventral_score']

    #############################
    # Save qs

    save_files_for_R_conversion(a, QS_TMP_FOLDER)
    build_qs_from_python_files(QS_TMP_FOLDER, output_qs_name=ct.replace(" ", "_"))

    #############################À
    # Compute umap

    preprocessing.preprocess(a, save_raw_counts=(not("counts" in a.layers)))
    #a.obs["Final_Zone_Assignments"] = a.obs["Final_Zone_Assignments"].astype("category")

    # remoc .X useless
    import scipy.sparse as sp
    a.X = sp.csr_matrix(a.shape)

    # SAVE BACK to original adata (only for these cells)
    cell_indices = [adata.obs_names.get_loc(cell) for cell in a.obs_names]
    adata.obsm['X_umap'][cell_indices, :] = a.obsm['X_umap']

    #############################À
    # Plot umap

    # Plot
    sc.pl.embedding(a, basis="X_umap", color=['gradient_score', 'dorsal_score', 'ventral_score'] , cmap='RdBu_r')# vmin=-100, vmax=100)
    sc.pl.embedding(a, basis="X_umap", color=[SAMPLE_VARIABLE, CONTRAST_VARIABLE, *COVARIATES_FOR_DEG, ])# vmin=-100, vmax=100)

    # Histograms
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    for ax, col, color, title in zip(axes, 
                                    ['gradient_score', 'dorsal_score', 'ventral_score'],
                                    ['purple', 'red', 'blue'],
                                    ['Combined', 'Dorsal', 'Ventral']):
        ax.hist(a.obs[col], bins=100, color=color, alpha=0.7)
        ax.set_xlabel(f'{title} Score')
        ax.axvline(0, color='black', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()

    # only if it has spatial:
    if "spatial" in adata.obsm:
        sc.pl.embedding(a, basis="spatial", color=["gradient_score", "ventral_score", "dorsal_score"])

    break

STRv_D1_MSN AnnData object with n_obs × n_vars = 26522 × 37905
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'donor_id', 'PREFIX', 'CELL_BARCODE', 'NUM_GENIC_READS', 'NUM_TRANSCRIPTS', 'NUM_GENES', 'num_retained_transcripts', 'pct_coding', 'pct_utr', 'pct_intergenic', 'pct_genic', 'pct_intronic', 'pct_mt', 'pct_ribosomal', 'frac_contamination', 'experiment', 'Neighborhood_name', 'Neighborhood_bootstrapping_probability', 'Class_name', 'Class_bootstrapping_probability', 'Subclass_name', 'Subclass_bootstrapping_probability', 'Group_name', 'Group_bootstrapping_probability', 'Cluster_name', 'Cluster_alias', 'Cluster_bootstrapping_probability', 'x', 'y', 'library', 'donor', 'x_um2', 'y_um2', 'unique_cell_ID', 'cb', 'Final_Zone_Assignments', 'Group_name.1', 'cell_type', 'Cohort', 'Biobank', 'Age.at.Death', 'Sex', 'Race', 'Ethnicity', 'Cause.of.Death', 'PMI', 'spn_type', 'case_control', 'RNA_snn_res.0.2', 'seurat_clusters', 'RNA_snn_res.0.5', 'Row.names', 'PC_1', 'PC_2', 'PC_3', 'PC_4',

2026-01-26 18:41:23 | [WARNING] R callback write-console: Loading required package: SeuratObject
  
2026-01-26 18:41:24 | [WARNING] R callback write-console: Loading required package: sp
  
2026-01-26 18:41:25 | [WARNING] R callback write-console: 
Attaching package: ‘SeuratObject’

  
2026-01-26 18:41:25 | [WARNING] R callback write-console: The following objects are masked from ‘package:base’:

    %||%, intersect, t

  



    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

2026-01-26 18:41:27 | [WARNING] R callback write-console: 
Attaching package: ‘Seurat’

  
2026-01-26 18:41:27 | [WARNING] R callback write-console: The following object is masked from ‘package:base’:

    %||%

  
2026-01-26 18:41:27 | [WARNING] R callback write-console: qs 0.27.3. Announcement: https://github.com/qsbase/qs/issues/103
  
2026-01-26 18:43:18 | [WARNING] R callback write-console: Warning:  
2026-01-26 18:43:18 | [WARNING] R callback write-console:  Data is of class dgTMatrix. Coercing to dgCMatrix.
  


# DEG

In [ ]:
for ct in adata.obs[CT_FOR_DEG_VARIABLE].unique():

    # Run Nebula
    nebula_result_path = DEG.run_nebula_parallel_script(
        path_qs = f"{QS_TMP_FOLDER}/{ct.replace(" ", "_")}.qs",
        path_nebula_script = PARALLEL_NEBULA_SCRIPT_PATH,
        id_col = SAMPLE_VARIABLE,
        covs = [CONTRAST_VARIABLE, *COVARIATES_FOR_DEG], # wihtout LIBRARY_SIZE_COL, with constrst vaibale too
        offset_col = LIBRARY_SIZE_COL,
        n_folds = 8,
        n_cores = 16,
        save_tmp = False,
        suffix = None,
    )

    break
    

Running command: bash /home/gdallagl/myworkdir/XDP/utils/nebula_scripts/run-nebula-parallel.sh --path /home/gdallagl/myworkdir/XDP/data/_old/nebula_tmp/R_obj.qs --id-col donor_id --offset-col nCount_RNA --n-folds 8 --n-cores 16 --save-tmp 0 --covs case_control,Age.at.Death,Sex,PMI,pct_mt,pct_intronic
STDOUT:
[INFO] Input: /home/gdallagl/myworkdir/XDP/data/_old/nebula_tmp/R_obj.qs
[INFO] Script dir: /home/gdallagl/myworkdir/XDP/utils/nebula_scripts
[INFO] Base name: R_obj.qs
[INFO] Geneset dir: /home/gdallagl/myworkdir/data/_old/nebula_tmp
[INFO] Geneset: nebula_tmp
[INFO] Split dir: /home/gdallagl/myworkdir/data/_old
[INFO] Split: _old
[INFO] n-folds: 8
[INFO] parallel jobs: 16
[INFO] id-col: donor_id
[INFO] covs:
       - case_control
       - Age.at.Death
       - Sex
       - PMI
       - pct_mt
       - pct_intronic
[INFO] offset-col: nCount_RNA  -> will create meta 'nUMI_nebula'
[INFO] Tmp dir: /home/gdallagl/myworkdir/data/_old/nebula_tmp/R_obj__nebula_ln
[INFO] Chunks dir: /home

In [3]:
for ct in adata.obs[CT_FOR_DEG_VARIABLE].unique():

    path_qs = f"{QS_TMP_FOLDER}/{ct.replace(' ', '_')}.qs"
    output_csv = f"{QS_TMP_FOLDER}/{ct.replace(' ', '_')}_nebula_results.csv"

    print(ct, "\n", path_qs, "\n", output_csv)
    
    import rpy2.robjects as ro

    ro.r(f"""
        library(nebula)
        library(qs)
        library(Seurat)
        
        # Load Seurat object
        cat("Loading", "{ct}", "...\\n")
        sobj <- qread("{path_qs}")
        
        # Extract count matrix (genes x cells) - Seurat v5
        count_matrix <- GetAssayData(sobj, assay = "RNA", layer = "counts")
        #count_matrix <- as.matrix(count_matrix)

        # ========== CRITICAL: FILTER GENES FIRST ==========
        # Remove genes with very low expression
        genes_total_counts <- Matrix::rowSums(count_matrix)
        genes_n_cells <- Matrix::rowSums(count_matrix > 0)
        
        # Keep only genes with:
            # - At least 50 cells expressing
            # - At least 100 total counts across all cells
        keep_genes <- (genes_n_cells >= 50) & (genes_total_counts >= 100)
        cat("Genes before filtering:", nrow(count_matrix), "\\n")
        count_matrix <- count_matrix[keep_genes, ]
        cat("Genes after filtering:", nrow(count_matrix), "\\n")
        
        
        # Extract metadata
        meta <- sobj@meta.data
        
        # CRITICAL: Ensure factors are properly coded
        # Convert condition to factor if it isn't already
        meta${CONTRAST_VARIABLE} <- as.factor(meta${CONTRAST_VARIABLE})
        
        # Get ID vector (must be factor for NEBULA)
        id <- as.factor(meta[["{SAMPLE_VARIABLE}"]])
        
        # Get offset (library size)
        offset <- meta[["{LIBRARY_SIZE_COL}"]]
        
        cat("\\nData dimensions:\\n")
        cat("  Genes:", nrow(count_matrix), "\\n")
        cat("  Cells:", ncol(count_matrix), "\\n")
        cat("  Subjects:", length(unique(id)), "\\n")
        cat("  Condition levels:", levels(meta${CONTRAST_VARIABLE}), "\\n")
        
        # Build DESIGN MATRIX (not just data frame!)
        # NEBULA needs model.matrix format with intercept
        pred_formula <- as.formula("~ {' + '.join([CONTRAST_VARIABLE, *COVARIATES_FOR_DEG])}")
        pred_matrix <- model.matrix(pred_formula, data = meta)
        
        cat("\\nPredictor matrix dimensions:", dim(pred_matrix), "\\n")
        cat("Predictor columns:", colnames(pred_matrix), "\\n")
        
        # Group cells by subject (recommended for speed and accuracy)
        # Try to group (reorder) cells by subject
        data_g <- group_cell(
            count = count_matrix,
            id = id,
            pred = pred_matrix,
            offset = offset
        )
        
        # If NULL, cells already sorted - use original data
        if (is.null(data_g)) {{
            cat("Cells already sorted by subject - using original data\\n")
            data_g <- list(
                count = count_matrix,
                id = id,
                pred = pred_matrix,
                offset = offset
            )
        }}
                
        # Run NEBULA with LN method (log-normal)
        cat("\\nRunning NEBULA LN method...\\n")
        re <- nebula(
            count = data_g$count,
            id = data_g$id,
            pred = data_g$pred,
            offset = data_g$offset,
            method = 'LN', # HL more accurate
            ncore = 4
        )
        
        cat("\\nNEBULA completed!\\n")
        
        # Extract summary
        summary_df <- re$summary
        
        cat("Result columns:", colnames(summary_df), "\\n")
        cat("Number of genes tested:", nrow(summary_df), "\\n")
        
        # Save full results
        write.csv(summary_df, "{output_csv}", row.names = FALSE, quote = FALSE)
        cat("Saved results to {output_csv}\\n")
        
        # Print summary statistics
        if ("{CONTRAST_VARIABLE}" %in% colnames(summary_df)) {{
            sig_genes <- sum(summary_df$p_{CONTRAST_VARIABLE} < 0.05, na.rm = TRUE)
            cat("Significant genes (p < 0.05):", sig_genes, "\\n")
        }}

        rm(sobj, count_matrix, meta, data_g, re)
        gc()
    """)
    
    print(f"✓ Completed {ct}\n")
    
    break

STRv_D1_MSN 
 /home/gdallagl/myworkdir/XDP/data/_old/nebula_tmp/STRv_D1_MSN.qs 
 /home/gdallagl/myworkdir/XDP/data/_old/nebula_tmp/STRv_D1_MSN_nebula_results.csv

    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

2026-01-27 01:22:20 | [WARNING] R callback write-console: qs 0.27.3. Announcement: https://github.com/qsbase/qs/issues/103
  
2026-01-27 01:22:20 | [WARNING] R callback write-console: Loading required package: SeuratObject
  
2026-01-27 01:22:20 | [WARNING] R callback write-console: Loading required package: sp
  
2026-01-27 01:22:20 | [WARNING] R callback write-console: 
Attaching package: ‘SeuratObject’

  
2026-01-27 01:22:20 | [WARNING] R callback write-console: The following objects are masked from ‘package:base’:

    %||%, intersect, t

  
2026-01-27 01:22:20 | [WARNING] R callback write-console: 
Attaching package: ‘Seurat’

  
2026-01-27 01:22:20 | [WARNING] R callback write-console: The following object is masked from ‘package:base’:

    %||%

  


Loading STRv_D1_MSN ...
Genes before filtering: 37905 
Genes after filtering: 28522 

Data dimensions:
  Genes: 28522 
  Cells: 26522 
  Subjects: 19 
  Condition levels: case control 

Predictor matrix dimensions: 26522 7 
Predictor columns: (Intercept) case_controlcontrol Age.at.Death SexMale PMI pct_mt pct_intronic 
The cells are already grouped.Cells already sorted by subject - using original data

Running NEBULA LN method...
Remove  951  genes having low expression.
Analyzing  27571  genes with  19  subjects and  26522  cells.

NEBULA completed!
Result columns: logFC_(Intercept) logFC_case_controlcontrol logFC_Age.at.Death logFC_SexMale logFC_PMI logFC_pct_mt logFC_pct_intronic se_(Intercept) se_case_controlcontrol se_Age.at.Death se_SexMale se_PMI se_pct_mt se_pct_intronic p_(Intercept) p_case_controlcontrol p_Age.at.Death p_SexMale p_PMI p_pct_mt p_pct_intronic gene_id gene 
Number of genes tested: 27571 
Saved results to /home/gdallagl/myworkdir/XDP/data/_old/nebula_tmp/STRv_D1

In [4]:
output_csv

'/home/gdallagl/myworkdir/XDP/data/_old/nebula_tmp/STRv_D1_MSN_nebula_results.csv'

In [6]:
df = pd.read_csv(output_csv)


from statsmodels.stats.multitest import multipletests
df["FDR_case_controlcontrol"] = multipletests(
    df["p_case_controlcontrol"],
    method="fdr_bh"
)[1]


print(df.columns)
df = df[["gene", "logFC_case_controlcontrol", "se_case_controlcontrol", "p_case_controlcontrol", "FDR_case_controlcontrol"]]

import numpy as np
import seaborn as sns

display(df)


Index(['logFC_(Intercept)', 'logFC_case_controlcontrol', 'logFC_Age.at.Death',
       'logFC_SexMale', 'logFC_PMI', 'logFC_pct_mt', 'logFC_pct_intronic',
       'se_(Intercept)', 'se_case_controlcontrol', 'se_Age.at.Death',
       'se_SexMale', 'se_PMI', 'se_pct_mt', 'se_pct_intronic', 'p_(Intercept)',
       'p_case_controlcontrol', 'p_Age.at.Death', 'p_SexMale', 'p_PMI',
       'p_pct_mt', 'p_pct_intronic', 'gene_id', 'gene',
       'FDR_case_controlcontrol'],
      dtype='object')


,gene,logFC_case_controlcontrol,se_case_controlcontrol,p_case_controlcontrol,FDR_case_controlcontrol
0,A1BG,0.063365,0.070833,0.371015,0.859736
1,A1BG-AS1,0.058595,0.058071,0.312962,0.846251
2,A1CF,0.246171,0.154522,0.111136,0.699734
3,A2M,-0.230577,0.192797,0.231713,0.812951
4,A2M-AS1,-0.015467,0.067757,0.819435,0.974565
...,...,...,...,...,...
27566,ZZEF1,0.015326,0.025268,0.544158,0.913761
27567,ZZZ3,-0.039904,0.050729,0.431501,0.885320
27568,ENSG00000205653,-0.131101,0.619296,0.832346,0.977431
27569,ENSG00000223438,0.048099,0.493776,0.922401,0.989732


In [7]:
# Read aritificla perturbed gene
    # checj min lofc
import pandas as pd
pert_de_results = pd.read_csv(f"/home/gdallagl/myworkdir/XDP/data/XDP/artificial_bican/geneset_001/genes.csv")
display(pert_de_results.head(3))

,gene_id,log2fc,frac_applied,n_subjects_applied,subjects_applied
0,ENSG00000237280,0.522222,0.3,3,"MD9129,MS986638,UMBEB23076"
1,ENSG00000250027,0.733333,0.7,7,"MD9129,MS30499,MS706984,MS876075,MS986638,UMBE..."
2,ENSG00000289321,-0.733333,0.4,4,"MD6927,MS30499,MS876075,MS986638"


In [8]:
de_results = df

de_results["is_significant"] = (de_results.FDR_case_controlcontrol <= PVAL_THR) & (np.abs(de_results.logFC_case_controlcontrol) >= LOGFC_THR)
de_results["is_true_DEG"] = de_results.index.isin(pert_de_results.gene_id.tolist())


display(de_results[de_results.is_significant == True])

# Confussion matrix
cm_df = pd.crosstab(de_results["is_true_DEG"], de_results["is_significant"], normalize="all")
sns.heatmap(cm_df, annot=True, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Calculate performance metrics
de_results["TP"] = (de_results["is_true_DEG"] == True)  & (de_results["is_significant"] == True)
de_results["TN"] = (de_results["is_true_DEG"] == False) & (de_results["is_significant"] == False)
de_results["FP"] = (de_results["is_true_DEG"] == False) & (de_results["is_significant"] == True)
de_results["FN"] = (de_results["is_true_DEG"] == True)  & (de_results["is_significant"] == False)
tp = de_results['TP'].sum()
tn = de_results['TN'].sum()
fp = de_results['FP'].sum()
fn = de_results['FN'].sum()

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (tp + tn) / (tp + tn + fp + fn)

print(f"\nPrecision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")
print(f"Accuracy:  {accuracy:.3f}")

,gene,logFC_case_controlcontrol,se_case_controlcontrol,p_case_controlcontrol,FDR_case_controlcontrol,is_significant,is_true_DEG
23,AARS1,-0.293307,0.079207,2.130443e-04,4.319004e-02,True,False
173,ACOT13,0.114739,0.024842,3.861126e-06,1.478543e-03,True,False
354,ADGRL3,0.747446,0.095594,5.324535e-15,1.468027e-11,True,False
1008,ARHGEF5,1.164079,0.287769,5.228034e-05,1.427150e-02,True,False
1233,ATG9B,0.634017,0.140377,6.285811e-06,2.221873e-03,True,False
...,...,...,...,...,...,...,...
26743,ZBTB5,0.151118,0.040514,1.914755e-04,3.993960e-02,True,False
26829,ZFAND3-DT,0.827735,0.205594,5.672067e-05,1.529272e-02,True,False
26856,ZFP62,-0.708656,0.127828,2.959400e-08,2.266489e-05,True,False
26882,ZFYVE27,-0.968281,0.159830,1.376608e-09,1.308774e-06,True,False



Precision: 0.000
Recall:    0.000
F1 Score:  0.000
Accuracy:  0.994
